#### Write a function to parse the HTML response, return the data we want(name, introduction, ...etc).


In [33]:
import requests
from bs4 import BeautifulSoup as bs

def process_document(document):
    
    nodes = document.select('dl#plist')
    data = list()

    for li in nodes:
        # get link
        li_link = li.select_one('a')['href']

        # get book name
        li_name = li.select_one('a').get_text()

        # get introduction
        li_introduction = li.select_one('.des').string.split()

        # request for document
        li_res = requests.get(li_link)
        li_doc = bs(li_res.text, 'lxml')        
        
        tmp_url = li_doc.select_one('div.down_2 > a')['href']
        tmp_url_res = requests.get(tmp_url)
        tmp_url_doc = bs(tmp_url_res.text, 'lxml')

        # get download url
        li_download = " "
        for ele in tmp_url_doc.select('span.downfile'):
            li_download = li_download + ele.select_one('a')['href'] + " , "

        # append new row
        data.append({
            'name' : li_name,
            'introduction': li_introduction,
            'download_link' : li_download,
        })
    return data

#### Observe that we can get e-books of different pagination with the link format except page is 1:http://www.zxcs.me/sort/28/page/2
#### We need define the page range we want.
#### Crawl over the news on the site, store the data in variable "all_data" .


In [34]:
start_page = 1
stop_page = 30
all_data = list()

for page in range(start_page, stop_page):
    print('start crawling page ' + str(page))
    if page == 1:
        res = requests.get('http://www.zxcs.me/sort/28')
    else :
        res = requests.get('http://www.zxcs.me/sort/28/page/' + str(page))
    doc = bs(res.text, 'lxml')
    data = process_document(doc)
    all_data += data




start crawling page 1
start crawling page 2
start crawling page 3
start crawling page 4
start crawling page 5
start crawling page 6
start crawling page 7
start crawling page 8
start crawling page 9
start crawling page 10
start crawling page 11
start crawling page 12
start crawling page 13
start crawling page 14
start crawling page 15
start crawling page 16
start crawling page 17
start crawling page 18
start crawling page 19
start crawling page 20
start crawling page 21
start crawling page 22
start crawling page 23
start crawling page 24
start crawling page 25
start crawling page 26
start crawling page 27
start crawling page 28
start crawling page 29


#### Check the result

In [35]:
all_data[0:5]

[{'name': '《天下豪商》（校对版全本）作者：大罗罗',
  'introduction': ['【TXT大小】：9.28',
   'MB',
   '【内容简介】：',
   '大宋元符年间，画师武浩来到了繁华似锦赵氏天下。',
   '走在宛如清明上河图般繁华的汴梁街头，武浩却想到了29年后，女真铁骑，席卷南下，将这烈火烹油一般的盛世景象，全都毁了个干干净净。',
   '可是一介布衣，纵然知晓大厦将倾，又哪来的挽天之力？',
   '只想着在大难之中独善其身的武浩，却在汴梁街...'],
  'download_link': ' http://d1.zxcs6.xyz/20/txhs,dl.rar , http://d2.zxcs6.xyz/20/txhs,dl.rar , '},
 {'name': '《马前卒》（校对版全本）作者：枪手1号',
  'introduction': ['【TXT大小】：12.9',
   'MB',
   '【内容简介】：',
   '不停地战斗，不停的杀敌，秦风是国家的悍将，军队的马前卒，永远冲锋在第一线，用赫赫战功书写着自己光荣的履历，但大变陡至，昔日的功臣，转眼之间却成了国家的罪人，人人得而诛之的叛徒，兄弟血洒疆场，部下反目成仇，情人苦恋虐心，一时之间，四面楚歌。',
   '这位昔日的国之悍将身份反转，开始了为自己而...'],
  'download_link': ' http://d1.zxcs6.xyz/20/mqz,qs1hao.rar , http://d2.zxcs6.xyz/20/mqz,qs1hao.rar , '},
 {'name': '《红楼之庶子风流》（校对版全本）作者：屋外风吹凉',
  'introduction': ['【TXT大小】：5.63',
   'MB',
   '【内容简介】：',
   '外科医生贾琮过劳而卒，魂穿荣府。',
   '谱一曲红楼幽梦，唱一世庶子风流。',
   '正版订阅书源，收集于网络！',
   '文本由河洛校对！'],
  'download_link': ' http://d1.zxcs6.xyz/20/hlzszfl,wwfl.rar , http://d2.zxcs6.xyz/20/hlzszfl,wwfl.rar , '},
 {'na

###### Save as pkl file

In [36]:
import pickle

with open('zxcs_crawler.pkl', 'wb') as f:
    pickle.dump(all_data, f)

##### Turn it into pandas dataframe

In [37]:
import pandas as pd
pd.DataFrame(all_data)[['name', 'introduction', 'download_link']].head()

,name,introduction,download_link
0,《天下豪商》（校对版全本）作者：大罗罗,"[【TXT大小】：9.28, MB, 【内容简介】：, 大宋元符年间，画师武浩来到了繁华似锦...","http://d1.zxcs6.xyz/20/txhs,dl.rar , http://d..."
1,《马前卒》（校对版全本）作者：枪手1号,"[【TXT大小】：12.9, MB, 【内容简介】：, 不停地战斗，不停的杀敌，秦风是国家的...","http://d1.zxcs6.xyz/20/mqz,qs1hao.rar , http:..."
2,《红楼之庶子风流》（校对版全本）作者：屋外风吹凉,"[【TXT大小】：5.63, MB, 【内容简介】：, 外科医生贾琮过劳而卒，魂穿荣府。, ...","http://d1.zxcs6.xyz/20/hlzszfl,wwfl.rar , htt..."
3,《宰执天下》（校对版全本）作者：cuslaa,"[【TXT大小】：15.0, MB, 【内容简介】：, 宰者宰相，执者执政。, 上辅君王，下...","http://d1.zxcs6.xyz/20/zzx,cuslaa.rar , http:..."
4,《崇祯聊天群》（校对版全本）作者：叫天,"[【TXT大小】：6.89, MB, 【内容简介】：, 崇祯皇帝被忽悠上吊丢江山，但是，这本...","http://d1.zxcs6.xyz/20/czltq,jt.rar , http://..."
